In [1]:
import json

from pathlib import Path

import pandas as pd
import numpy as np

import biopsykit as bp

import matplotlib.pyplot as plt

from pepbench.io import load_challenge_results_from_folder
from pepbench.io import convert_hz_to_ms
from pepbench.io import impute_missing_values
from pepbench.datasets import EmpkinsDataset, GuardianDataset

#%matplotlib widget
%load_ext autoreload
%autoreload 2

In [2]:
root_path = Path("../../../")
print(root_path.absolute())

/home/woody/iwso/iwso173h/pepbench/experiments/pep_algorithm_benchmarking/notebooks/ML_analysis/Preprocessing/../../..


In [3]:
result_path = root_path.joinpath("results")
result_path

PosixPath('../../../results')

In [4]:
deploy_type = "local"

config_dict = json.load(root_path.joinpath("config.json").open(encoding="utf-8"))

empkins_base_path = Path(config_dict[deploy_type]["empkins_path"])
guardian_base_path = Path(config_dict[deploy_type]["guardian_path"])
print(empkins_base_path)

/Users/sebas/Development/ResearchInternship/Data/2024_08_PEP_Benchmarking/EmpkinS_Dataset


#### Set flags for further processing

In [5]:
save_results = True
include_rr_interval = False
include_q_wave_reference = False
drop_nan = False
impute_nan = False

In [6]:
dataset_empkins = EmpkinsDataset(empkins_base_path, use_cache=True, only_labeled=True)
fs_empkins = dataset_empkins.sampling_rate_icg
print(f"Sampling rate ICG: {fs_empkins}")
dataset_guardian = GuardianDataset(guardian_base_path, use_cache=True, only_labeled=True)
fs_guardian = dataset_guardian.sampling_rate_icg
print(f"Sampling rate ICG: {fs_guardian}")

Sampling rate ICG: 1000
Sampling rate ICG: 500


In [7]:
algo_levels = ["q_wave_algorithm", "b_point_algorithm", "outlier_correction_algorithm"]
algo_level_mapping = dict(zip(algo_levels, ["Q-Wave Algorithm", "B-Point Algorithm", "Outlier Correction"]))

#### Load data from the challenge results

In [8]:
results_empkins = load_challenge_results_from_folder(
    result_path.joinpath("empkins_dataset_b_point").resolve(), index_cols_per_sample=["participant", "condition", "phase"]
).per_sample
results_guardian = load_challenge_results_from_folder(
    result_path.joinpath("guardian_dataset_b_point").resolve(), index_cols_per_sample=["participant", "phase"]
).per_sample

In [9]:
results_empkins = results_empkins.reset_index().set_index(['q_peak_algorithm', 'b_point_algorithm', 'outlier_correction_algorithm', 'participant', 'condition', 'phase'])
#results_empkins = results_empkins.drop(columns=['level_6'])
results_empkins

id  \
                                                                                                                  
q_peak_algorithm b_point_algorithm               outlier_correction_algorithm participant condition phase         
q-peak-reference arbol2017-isoelectric-crossings forouzanfar2018              VP_001      tsst      Prep      0   
                                                                                                    Prep      1   
                                                                                                    Prep      2   
                                                                                                    Prep      3   
                                                                                                    Prep      4   
...                                                                                                          ..   
                 stern1985                       none                         VP_032      ftsst     Pause_5   7   
                                                                                                    Pause_5   8   
                                                                                                    Pause_5   9   
                                                                                                    Pause_5  10   
                                                                                                    Pause_5  11   

                                                                                                            heartbeat_id  \
                                                                                                               estimated   
q_peak_algorithm b_point_algorithm               outlier_correction_algorithm participant condition phase                  
q-peak-reference arbol2017-isoelectric-crossings forouzanfar2018              VP_001      tsst      Prep               0   
                                                                                                    Prep               1   
                                                                                                    Prep               2   
                                                                                                    Prep               3   
                                                                                                    Prep               4   
...                                                                                                                  ...   
                 stern1985                       none                         VP_032      ftsst     Pause_5            7   
                                                                                                    Pause_5            8   
                                                                                                    Pause_5            9   
                                                                                                    Pause_5           10   
                                                                                                    Pause_5           11   

                                                                                                                       \
                                                                                                            reference   
q_peak_algorithm b_point_algorithm               outlier_correction_algorithm participant condition phase               
q-peak-reference arbol2017-isoelectric-crossings forouzanfar2018              VP_001      tsst      Prep            0   
                                                                                                    Prep            1   
                                                                                                    Prep            2   
                                   

#### Introduce column 'condition' to the guardian dataset to assure compatibility with the empkins dataset

In [10]:
results_guardian = results_guardian.assign(condition="Dummy")
results_guardian = results_guardian.reset_index().set_index(['q_peak_algorithm', 'b_point_algorithm', 'outlier_correction_algorithm', 'participant', 'condition', 'phase'])
results_guardian

id  \
                                                                                                                   
q_peak_algorithm b_point_algorithm               outlier_correction_algorithm participant condition phase          
q-peak-reference arbol2017-isoelectric-crossings forouzanfar2018              GDN0005     Dummy     Pause      0   
                                                                                                    Pause      1   
                                                                                                    Pause      2   
                                                                                                    Pause      3   
                                                                                                    Pause      4   
...                                                                                                           ..   
                 stern1985                       none                         GDN0030     Dummy     TiltDown  51   
                                                                                                    TiltDown  52   
                                                                                                    TiltDown  53   
                                                                                                    TiltDown  54   
                                                                                                    TiltDown  55   

                                                                                                             heartbeat_id  \
                                                                                                                estimated   
q_peak_algorithm b_point_algorithm               outlier_correction_algorithm participant condition phase                   
q-peak-reference arbol2017-isoelectric-crossings forouzanfar2018              GDN0005     Dummy     Pause               0   
                                                                                                    Pause               1   
                                                                                                    Pause               2   
                                                                                                    Pause               3   
                                                                                                    Pause               4   
...                                                                                                                   ...   
                 stern1985                       none                         GDN0030     Dummy     TiltDown           51   
                                                                                                    TiltDown           52   
                                                                                                    TiltDown           53   
                                                                                                    TiltDown           54   
                                                                                                    TiltDown           55   

                                                                                                                        \
                                                                                                             reference   
q_peak_algorithm b_point_algorithm               outlier_correction_algorithm participant condition phase                
q-peak-reference arbol2017-isoelectric-crossings forouzanfar2018              GDN0005     Dummy     Pause            0   
                                                                                                    Pause            1   
                                                                                                    Pause            2   
  

#### Perform data preprocessing

In [11]:
results_empkins_norm = results_empkins.copy()
results_guardian_norm = results_guardian.copy()
results_empkins_norm[("b_point_sample", "estimated")] = results_empkins[("b_point_sample", "estimated")] - results_empkins[("heartbeat_start_sample", "estimated")]
results_empkins_norm[("b_point_sample", "reference")] = results_empkins[("b_point_sample", "reference")] - results_empkins[("heartbeat_start_sample", "estimated")]
results_empkins_norm[("q_peak_sample", "reference")] = results_empkins[("q_peak_sample", "reference")] - results_empkins[("heartbeat_start_sample", "estimated")]
results_guardian_norm[("b_point_sample", "estimated")] = results_guardian[("b_point_sample", "estimated")] - results_guardian[("heartbeat_start_sample", "estimated")]
results_guardian_norm[("b_point_sample", "reference")] = results_guardian[("b_point_sample", "reference")] - results_guardian[("heartbeat_start_sample", "estimated")]
results_guardian_norm[("q_peak_sample", "reference")] = results_guardian[("q_peak_sample", "reference")] - results_guardian[("heartbeat_start_sample", "estimated")]

In [12]:
cols = [("heartbeat_start_sample", "estimated"), ("heartbeat_start_sample", "reference"), ("heartbeat_end_sample", "estimated"), ("heartbeat_end_sample", "reference"), ("q_peak_sample", "estimated"), ("q_peak_sample", "reference"), ("b_point_sample", "estimated"), ("b_point_sample", "reference"), ("pep_sample", "estimated"), ("pep_sample", "reference")]
results_empkins_cleaned_ms = results_empkins_norm.copy()
results_guardian_cleaned_ms = results_guardian_norm.copy()
results_empkins_cleaned_ms[cols] = results_empkins_norm[cols] * convert_hz_to_ms(sampling_frequency=fs_empkins)
results_guardian_cleaned_ms[cols] = results_guardian_norm[cols] * convert_hz_to_ms(sampling_frequency=fs_guardian)

#### Concatenate the empkins and guardian dataframes

In [13]:
preprocessed_results_empkins_guardian = pd.concat([results_empkins_cleaned_ms, results_guardian_cleaned_ms])
preprocessed_results_empkins_guardian

id  \
                                                                                                                   
q_peak_algorithm b_point_algorithm               outlier_correction_algorithm participant condition phase          
q-peak-reference arbol2017-isoelectric-crossings forouzanfar2018              VP_001      tsst      Prep       0   
                                                                                                    Prep       1   
                                                                                                    Prep       2   
                                                                                                    Prep       3   
                                                                                                    Prep       4   
...                                                                                                           ..   
                 stern1985                       none                         GDN0030     Dummy     TiltDown  51   
                                                                                                    TiltDown  52   
                                                                                                    TiltDown  53   
                                                                                                    TiltDown  54   
                                                                                                    TiltDown  55   

                                                                                                             heartbeat_id  \
                                                                                                                estimated   
q_peak_algorithm b_point_algorithm               outlier_correction_algorithm participant condition phase                   
q-peak-reference arbol2017-isoelectric-crossings forouzanfar2018              VP_001      tsst      Prep                0   
                                                                                                    Prep                1   
                                                                                                    Prep                2   
                                                                                                    Prep                3   
                                                                                                    Prep                4   
...                                                                                                                   ...   
                 stern1985                       none                         GDN0030     Dummy     TiltDown           51   
                                                                                                    TiltDown           52   
                                                                                                    TiltDown           53   
                                                                                                    TiltDown           54   
                                                                                                    TiltDown           55   

                                                                                                                        \
                                                                                                             reference   
q_peak_algorithm b_point_algorithm               outlier_correction_algorithm participant condition phase                
q-peak-reference arbol2017-isoelectric-crossings forouzanfar2018              VP_001      tsst      Prep             0   
                                                                                                    Prep             1   
                                                                                                    Prep             2   
  

In [14]:
input_data = preprocessed_results_empkins_guardian.reset_index().set_index(['q_peak_algorithm', 'b_point_algorithm', 'outlier_correction_algorithm', 'participant', 'condition', 'phase'])
input_data = input_data.xs(key='none', level='outlier_correction_algorithm', drop_level=True)
input_data

id  \
                                                                                      
q_peak_algorithm b_point_algorithm               participant condition phase          
q-peak-reference arbol2017-isoelectric-crossings VP_001      tsst      Prep       0   
                                                                       Prep       1   
                                                                       Prep       2   
                                                                       Prep       3   
                                                                       Prep       4   
...                                                                              ..   
                 stern1985                       GDN0030     Dummy     TiltDown  51   
                                                                       TiltDown  52   
                                                                       TiltDown  53   
                                                                       TiltDown  54   
                                                                       TiltDown  55   

                                                                                heartbeat_id  \
                                                                                   estimated   
q_peak_algorithm b_point_algorithm               participant condition phase                   
q-peak-reference arbol2017-isoelectric-crossings VP_001      tsst      Prep                0   
                                                                       Prep                1   
                                                                       Prep                2   
                                                                       Prep                3   
                                                                       Prep                4   
...                                                                                      ...   
                 stern1985                       GDN0030     Dummy     TiltDown           51   
                                                                       TiltDown           52   
                                                                       TiltDown           53   
                                                                       TiltDown           54   
                                                                       TiltDown           55   

                                                                                           \
                                                                                reference   
q_peak_algorithm b_point_algorithm               participant condition phase                
q-peak-reference arbol2017-isoelectric-crossings VP_001      tsst      Prep             0   
                                                                       Prep             1   
                                                                       Prep             2   
                                                                       Prep             3   
                                                                       Prep             4   
...                                                                                   ...   
                 stern1985                       GDN0030     Dummy     TiltDown        51   
                                                                       TiltDown        52   
                                                                       TiltDown        53   
                                                                       TiltDown        54   
                                                                       TiltDown        55   

                                                                                heartbeat_start_sample  \
                                                                                             estimated   
q_

#### Select columns

In [15]:
if include_rr_interval:
    input_data = input_data[[("heartbeat_id", "reference"), ("b_point_sample", "estimated"), ("b_point_sample", "reference"), ("rr_interval_ms", "estimated")]]
elif include_q_wave_reference:
    input_data = input_data[[("heartbeat_id", "reference"), ("b_point_sample", "estimated"), ("b_point_sample", "reference"), ("rr_interval_ms", "estimated"), ("q_wave_onset_sample", "reference")]]
else:
    input_data = input_data[[("heartbeat_id", "reference"), ("b_point_sample", "estimated"), ("b_point_sample", "reference")]]
input_data.columns = input_data.columns.to_flat_index().str.join("_")
input_data

heartbeat_id_reference  \
q_peak_algorithm b_point_algorithm               participant condition phase                              
q-peak-reference arbol2017-isoelectric-crossings VP_001      tsst      Prep                           0   
                                                                       Prep                           1   
                                                                       Prep                           2   
                                                                       Prep                           3   
                                                                       Prep                           4   
...                                                                                                 ...   
                 stern1985                       GDN0030     Dummy     TiltDown                      51   
                                                                       TiltDown                      52   
                                                                       TiltDown                      53   
                                                                       TiltDown                      54   
                                                                       TiltDown                      55   

                                                                                 b_point_sample_estimated  \
q_peak_algorithm b_point_algorithm               participant condition phase                                
q-peak-reference arbol2017-isoelectric-crossings VP_001      tsst      Prep                         275.0   
                                                                       Prep                         254.0   
                                                                       Prep                         265.0   
                                                                       Prep                         260.0   
                                                                       Prep                         259.0   
...                                                                                                   ...   
                 stern1985                       GDN0030     Dummy     TiltDown                     496.0   
                                                                       TiltDown                     436.0   
                                                                       TiltDown                     454.0   
                                                                       TiltDown                     388.0   
                                                                       TiltDown                     394.0   

                                                                                 b_point_sample_reference  
q_peak_algorithm b_point_algorithm               participant condition phase                               
q-peak-reference arbol2017-isoelectric-crossings VP_001      tsst      Prep                         240.0  
                                                                       Prep                         222.0  
                                                                       Prep                         242.0  
                                                                       Prep                         248.0  
                                                                       Prep                         242.0  
...                                                                                                   ...  
                 stern1985                       GDN0030     Dummy     TiltDown                     498.0  
                                                                       TiltDown                     490.0  
                                                                       TiltDown                     454.0  
                                                                       TiltDown    

In [16]:
print(f"Amount of algorithms: {len(input_data.index.get_level_values('b_point_algorithm').unique())}")

Amount of algorithms: 10


#### Transform dataframe from long to wide format

In [17]:
input_data = input_data.reset_index()
# Pivot the DataFrame
if include_rr_interval:
    input_data_wide = input_data.pivot(
        index=['participant', 'condition', 'phase', 'heartbeat_id_reference', 'b_point_sample_reference', 'rr_interval_ms_estimated'],
        columns='b_point_algorithm',
        values='b_point_sample_estimated'
    ).reset_index()
elif include_q_wave_reference:
    input_data_wide = input_data.pivot(
        index=['participant', 'condition', 'phase', 'heartbeat_id_reference', 'b_point_sample_reference', 'q_wave_onset_sample_reference'],
        columns='b_point_algorithm',
        values='b_point_sample_estimated'
    ).reset_index()
else:
    input_data_wide = input_data.pivot(
        index=['participant', 'condition', 'phase', 'heartbeat_id_reference', 'b_point_sample_reference'],
        columns='b_point_algorithm',
        values='b_point_sample_estimated'
    ).reset_index()

input_data_wide

b_point_algorithm,participant,condition,phase,heartbeat_id_reference,b_point_sample_reference,arbol2017-isoelectric-crossings,arbol2017-second-derivative,arbol2017-third-derivative,debski1993-second-derivative,drost2022,forouzanfar2018,lozano2007-linear-regression,lozano2007-quadratic-regression,sherwood1990,stern1985
0,GDN0005,Dummy,HoldingBreath,0,388.0,438.0,398.0,394.0,452.0,400.0,550.0,412.0,384.0,442.0,388.0
1,GDN0005,Dummy,HoldingBreath,1,404.0,340.0,350.0,244.0,388.0,420.0,402.0,404.0,384.0,330.0,402.0
2,GDN0005,Dummy,HoldingBreath,3,376.0,382.0,296.0,386.0,366.0,386.0,388.0,366.0,348.0,382.0,374.0
3,GDN0005,Dummy,HoldingBreath,4,390.0,394.0,344.0,396.0,376.0,396.0,398.0,372.0,348.0,394.0,388.0
4,GDN0005,Dummy,HoldingBreath,5,386.0,398.0,312.0,388.0,418.0,392.0,390.0,378.0,354.0,400.0,384.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11606,VP_032,tsst,Talk,39,310.0,335.0,276.0,324.0,300.0,318.0,306.0,305.0,294.0,337.0,306.0
11607,VP_032,tsst,Talk,40,322.0,331.0,298.0,330.0,311.0,322.0,311.0,321.0,303.0,329.0,287.0
11608,VP_032,tsst,Talk,41,340.0,317.0,300.0,348.0,330.0,343.0,309.0,332.0,322.0,311.0,287.0
11609,VP_032,tsst,Talk,42,311.0,365.0,324.0,366.0,347.0,354.0,305.0,351.0,337.0,368.0,305.0


#### Drop nan values

In [18]:
print(f"Drop nan values: {drop_nan}")
print(f"Impute nan values: {impute_nan}")

Drop nan values: False
Impute nan values: False


#### Delete rows where the column 'rr_interval_ms_estimated' contains nan values

In [19]:
if include_rr_interval and drop_nan:
    input_data_wide = input_data_wide.dropna(subset=['rr_interval_ms_estimated'])
input_data_wide

b_point_algorithm,participant,condition,phase,heartbeat_id_reference,b_point_sample_reference,arbol2017-isoelectric-crossings,arbol2017-second-derivative,arbol2017-third-derivative,debski1993-second-derivative,drost2022,forouzanfar2018,lozano2007-linear-regression,lozano2007-quadratic-regression,sherwood1990,stern1985
0,GDN0005,Dummy,HoldingBreath,0,388.0,438.0,398.0,394.0,452.0,400.0,550.0,412.0,384.0,442.0,388.0
1,GDN0005,Dummy,HoldingBreath,1,404.0,340.0,350.0,244.0,388.0,420.0,402.0,404.0,384.0,330.0,402.0
2,GDN0005,Dummy,HoldingBreath,3,376.0,382.0,296.0,386.0,366.0,386.0,388.0,366.0,348.0,382.0,374.0
3,GDN0005,Dummy,HoldingBreath,4,390.0,394.0,344.0,396.0,376.0,396.0,398.0,372.0,348.0,394.0,388.0
4,GDN0005,Dummy,HoldingBreath,5,386.0,398.0,312.0,388.0,418.0,392.0,390.0,378.0,354.0,400.0,384.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11606,VP_032,tsst,Talk,39,310.0,335.0,276.0,324.0,300.0,318.0,306.0,305.0,294.0,337.0,306.0
11607,VP_032,tsst,Talk,40,322.0,331.0,298.0,330.0,311.0,322.0,311.0,321.0,303.0,329.0,287.0
11608,VP_032,tsst,Talk,41,340.0,317.0,300.0,348.0,330.0,343.0,309.0,332.0,322.0,311.0,287.0
11609,VP_032,tsst,Talk,42,311.0,365.0,324.0,366.0,347.0,354.0,305.0,351.0,337.0,368.0,305.0


In [20]:
if include_rr_interval:
    input_data_wide_cleaned = input_data_wide.reset_index().set_index(['participant', 'condition', 'phase', 'heartbeat_id_reference', 'b_point_sample_reference', 'rr_interval_ms_estimated'], append=True)
else:
    input_data_wide_cleaned = input_data_wide.reset_index().set_index(['participant', 'condition', 'phase', 'heartbeat_id_reference', 'b_point_sample_reference'], append=True)
input_data_wide_cleaned.index

MultiIndex([(    0, 'GDN0005', 'Dummy', 'HoldingBreath',  0, 388.0),
            (    1, 'GDN0005', 'Dummy', 'HoldingBreath',  1, 404.0),
            (    2, 'GDN0005', 'Dummy', 'HoldingBreath',  3, 376.0),
            (    3, 'GDN0005', 'Dummy', 'HoldingBreath',  4, 390.0),
            (    4, 'GDN0005', 'Dummy', 'HoldingBreath',  5, 386.0),
            (    5, 'GDN0005', 'Dummy', 'HoldingBreath',  6, 390.0),
            (    6, 'GDN0005', 'Dummy', 'HoldingBreath',  7, 388.0),
            (    7, 'GDN0005', 'Dummy', 'HoldingBreath',  9, 416.0),
            (    8, 'GDN0005', 'Dummy', 'HoldingBreath', 10, 378.0),
            (    9, 'GDN0005', 'Dummy', 'HoldingBreath', 11, 358.0),
            ...
            (11601,  'VP_032',  'tsst',          'Talk', 34, 323.0),
            (11602,  'VP_032',  'tsst',          'Talk', 35, 321.0),
            (11603,  'VP_032',  'tsst',          'Talk', 36, 284.0),
            (11604,  'VP_032',  'tsst',          'Talk', 37, 323.0),
            (11605

In [21]:
if drop_nan:
    input_data_wide_cleaned = input_data_wide.dropna()
else:
    if impute_nan:
        input_data_wide_cleaned = input_data_wide_cleaned.drop(columns=['index'])
        input_data_wide_cleaned = impute_missing_values(input_data_wide_cleaned, mode='median')
        print(f"Amount of rows after imputation: {input_data_wide_cleaned.shape[0]}")
        input_data_wide_cleaned = input_data_wide_cleaned.dropna()
        print(f"Amount of rows after deleting all nan rows: {input_data_wide_cleaned.shape[0]}")
    else:
        input_data_wide_cleaned = input_data_wide_cleaned.dropna(how='all')
input_data_wide_cleaned

,,,,,b_point_algorithm,index,arbol2017-isoelectric-crossings,arbol2017-second-derivative,arbol2017-third-derivative,debski1993-second-derivative,drost2022,forouzanfar2018,lozano2007-linear-regression,lozano2007-quadratic-regression,sherwood1990,stern1985
,participant,condition,phase,heartbeat_id_reference,b_point_sample_reference,,,,,,,,,,,
0,GDN0005,Dummy,HoldingBreath,0,388.0,0,438.0,398.0,394.0,452.0,400.0,550.0,412.0,384.0,442.0,388.0
1,GDN0005,Dummy,HoldingBreath,1,404.0,1,340.0,350.0,244.0,388.0,420.0,402.0,404.0,384.0,330.0,402.0
2,GDN0005,Dummy,HoldingBreath,3,376.0,2,382.0,296.0,386.0,366.0,386.0,388.0,366.0,348.0,382.0,374.0
3,GDN0005,Dummy,HoldingBreath,4,390.0,3,394.0,344.0,396.0,376.0,396.0,398.0,372.0,348.0,394.0,388.0
4,GDN0005,Dummy,HoldingBreath,5,386.0,4,398.0,312.0,388.0,418.0,392.0,390.0,378.0,354.0,400.0,384.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11606,VP_032,tsst,Talk,39,310.0,11606,335.0,276.0,324.0,300.0,318.0,306.0,305.0,294.0,337.0,306.0
11607,VP_032,tsst,Talk,40,322.0,11607,331.0,298.0,330.0,311.0,322.0,311.0,321.0,303.0,329.0,287.0
11608,VP_032,tsst,Talk,41,340.0,11608,317.0,300.0,348.0,330.0,343.0,309.0,332.0,322.0,311.0,287.0


In [22]:
input_data_wide_cleaned = input_data_wide_cleaned.drop(columns=['index'])
input_data_wide_cleaned

,,,,,b_point_algorithm,arbol2017-isoelectric-crossings,arbol2017-second-derivative,arbol2017-third-derivative,debski1993-second-derivative,drost2022,forouzanfar2018,lozano2007-linear-regression,lozano2007-quadratic-regression,sherwood1990,stern1985
,participant,condition,phase,heartbeat_id_reference,b_point_sample_reference,,,,,,,,,,
0,GDN0005,Dummy,HoldingBreath,0,388.0,438.0,398.0,394.0,452.0,400.0,550.0,412.0,384.0,442.0,388.0
1,GDN0005,Dummy,HoldingBreath,1,404.0,340.0,350.0,244.0,388.0,420.0,402.0,404.0,384.0,330.0,402.0
2,GDN0005,Dummy,HoldingBreath,3,376.0,382.0,296.0,386.0,366.0,386.0,388.0,366.0,348.0,382.0,374.0
3,GDN0005,Dummy,HoldingBreath,4,390.0,394.0,344.0,396.0,376.0,396.0,398.0,372.0,348.0,394.0,388.0
4,GDN0005,Dummy,HoldingBreath,5,386.0,398.0,312.0,388.0,418.0,392.0,390.0,378.0,354.0,400.0,384.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11606,VP_032,tsst,Talk,39,310.0,335.0,276.0,324.0,300.0,318.0,306.0,305.0,294.0,337.0,306.0
11607,VP_032,tsst,Talk,40,322.0,331.0,298.0,330.0,311.0,322.0,311.0,321.0,303.0,329.0,287.0
11608,VP_032,tsst,Talk,41,340.0,317.0,300.0,348.0,330.0,343.0,309.0,332.0,322.0,311.0,287.0


#### Save the preprocessed data to use it for training

In [23]:
print(f"Save results: {save_results}")
print(f"Include RR-Interval: {include_rr_interval}")
print(f"Drop missing values: {drop_nan}")
print(f"Impute missing values: {impute_nan}")
print(f"Include Q-Wave Reference: {include_q_wave_reference}")

Save results: True
Include RR-Interval: False
Drop missing values: False
Impute missing values: False
Include Q-Wave Reference: False


In [24]:
if save_results:
    if drop_nan:
        input_data_wide_cleaned.to_csv(result_path.joinpath("data/b-point/without-rr-interval/train_data_b_point.csv"))
    elif impute_nan:
        input_data_wide_cleaned.to_csv(result_path.joinpath("data/b-point/rr-interval/train_data_b_point_rr_interval_median_imputed.csv"))
    else:
        input_data_wide_cleaned.to_csv(result_path.joinpath("data/b-point/without-rr-interval/train_data_b_point_include_nan.csv"))
    print(f"Data saved!")

Data saved!


In [25]:
input_data_b_point = pd.read_csv(result_path.joinpath("data/b-point/without-rr-interval/train_data_b_point_include_nan.csv"), index_col=[0,1,2,3,4,5])
input_data_b_point

,,,,,,arbol2017-isoelectric-crossings,arbol2017-second-derivative,arbol2017-third-derivative,debski1993-second-derivative,drost2022,forouzanfar2018,lozano2007-linear-regression,lozano2007-quadratic-regression,sherwood1990,stern1985
,participant,condition,phase,heartbeat_id_reference,b_point_sample_reference,,,,,,,,,,
0,GDN0005,Dummy,HoldingBreath,0,388.0,438.0,398.0,394.0,452.0,400.0,550.0,412.0,384.0,442.0,388.0
1,GDN0005,Dummy,HoldingBreath,1,404.0,340.0,350.0,244.0,388.0,420.0,402.0,404.0,384.0,330.0,402.0
2,GDN0005,Dummy,HoldingBreath,3,376.0,382.0,296.0,386.0,366.0,386.0,388.0,366.0,348.0,382.0,374.0
3,GDN0005,Dummy,HoldingBreath,4,390.0,394.0,344.0,396.0,376.0,396.0,398.0,372.0,348.0,394.0,388.0
4,GDN0005,Dummy,HoldingBreath,5,386.0,398.0,312.0,388.0,418.0,392.0,390.0,378.0,354.0,400.0,384.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11606,VP_032,tsst,Talk,39,310.0,335.0,276.0,324.0,300.0,318.0,306.0,305.0,294.0,337.0,306.0
11607,VP_032,tsst,Talk,40,322.0,331.0,298.0,330.0,311.0,322.0,311.0,321.0,303.0,329.0,287.0
11608,VP_032,tsst,Talk,41,340.0,317.0,300.0,348.0,330.0,343.0,309.0,332.0,322.0,311.0,287.0


In [46]:
input_data_b_point['rr_interval_ms_estimated'].isna().any()

np.True_